# Numpy

[Numpy](https://www.numpy.org/) is the foundation of most of what you will do with scientific python. Modules like [scipy](https://www.scipy.org/) and [pandas](https://pandas.pydata.org/) are built on top of numpy, and it is the linga franca for most numerical work in python.

In [ ]:
import numpy as np

When you are first introduced to python, one of the big selling points is that it isn't statically typed. You can do things like

In [ ]:
a = 1
a = 'apple'
print(a)

And you don't get complaints from python about a being an integer. This is a _big_ advantage for python, and it works with collections as well

In [ ]:
myList = [1, 3., 'Ian', [range(3)], {'not': 'a good idea'}]
myList

Lists in python are about as general as they could be. This is very flexible and it lets you do some fancy things, but it has a price. The Python interpreter can't make any assumptions about what will come next in a list; everything has to be treated as a generic object. Python does a good job of hiding the complexity of doing this, but as the lists get longer and more complex the overhead gets larger, and eventually perfomance becomes unacceptable.

One solution to this is to use a statically typed language like C[<sup>1</sup>](#fn1 "footnote and tooltip 1"). There the burden of figuring out object types is left to the programmer, and the compiler can be much more efficient operating on them. A good example would be an array of `double`s. In memory, these will be allocated contiguously so when you need to jump to the 1402th double, you can do it with very simple arithmetic. Python has a much harder time because the memory allocated for your list could be a horrible mixture of all the different things you've stuffed in there. 

## The `ndarray`

Numpy attempts let you keep the advantages of Python without sacraficing the speed of static typing by adding the concept of homogenous collections to python: `ndarray`s. The `ndarray` is the foundational concept in numpy, it is an array object which represents a multidimensional, homogeneous array of fixed-size items and most commonly these items will be numbers.

In [ ]:
%%timeit
for i in range(1000000):
    i*i

In [ ]:
%timeit np.arange(1000000)**2

`ndarray`s look like python lists, but they are fundamentally different, e.g.

In [ ]:
a = [1, 2, 3, 4]
b = [5, 6, 7, 8]
a+b

In [ ]:
na = np.array([1, 2, 3, 4])
nb = np.array([5, 6, 7, 8])
na + nb

`numpy` was able to assume that the things in the `ndarray` were the compatible types and vectorize the addition, if we want the same thing with python lists, we have to jump through some hoops

In [ ]:
[ i + j for i, j in zip(a, b) ] 

Let's take a look at what makes up an `ndarray`.

In [ ]:
na = np.array([1,2,3,4,5])
na

In [ ]:
attr_names = [attr_name for attr_name in dir(np.ndarray)
              if not callable(getattr(np.ndarray, attr_name)) 
              and not attr_name[:2] == '__']

attr_help = {a : getattr(np.ndarray, a).__doc__.split('\n')[0] for a in attr_names}

In [ ]:
attr_help

By knowing these attributes, numpy can use some of the same tricks that statically typed languges use because the Python interpreter can now infer the memory layout.

numpy has some convenience methods for creating new ndarrays. As you saw above, one way to create an ndarray is to pass a list with the values to `np.array`. Here are some others...

Using np.array directly

In [ ]:
a1 = np.array([0, 1, 1, 2, 3, 5, 8, 13])

`np.arange` will generate numbers between limits

In [ ]:
a2 = np.arange(0, 100)
a2

In [ ]:
a22 = np.arange(0, 10, dtype=float)
a22

`np.linspace` is extremely useful, you tell it where to start and stop and how many samples you need and linspace does the rest. Here we will create numbers 100 numbers between 0 (inclusive) and 1, linearly spaced

In [ ]:
li1 = np.linspace(0, 1, 20)
li1

The `np.zeros` function will generate an `ndarray` full of zeros. The first argument is the shape which you can give as an integer (for 1d arrays) or a sequence (for n-dimensional arrays). You can also pass the `dtype=` argument to tell it what type of number you are looking for.

In [ ]:
# z1 100 integer zeros
z1 = np.zeros(100, dtype=int)

# z2 a 5,5 array of float64 zeros
z2 = np.zeros((5, 5), dtype=float)
z2.shape

The `np.ones` function does something similar, but with unit value

In [ ]:
# o1 100 integers ones
o1 = np.ones(100, dtype='float64')
o1

In [ ]:
# o2 a 5x5 ndarray of complex128 one values. First argument is shape sequence
o2 = np.ones((5, 5), dtype='complex128')
o2

The `np.eye` function generates a 2D array with ones down the diagonal, zeros elsewhere

In [ ]:
# e1 a 5x5 array with ones down the diagonal
e1 = np.eye(5, dtype=np.float64)
e1

We'll come back to the `numpy.random` module later on, but it has some useful convenience methods. `np.random.randint` returns random integers. Take a look at the help on the method then create a name `r1` with an array of `4x5` random numbers between `0` and `10`.


In [ ]:
# r1 a 4x5 array of random integers between 0 & 10, 
np.random.randint(0, 10, size=(4, 5))

## Indexing and Slicing

Now that we have some `ndarray`s to play with, lets look at using them. Of course, `ndarray`s are zero indexed


In [ ]:
# First element of a2
a2[0]

The usual negative number notation works for counting from the end

In [ ]:
# 2nd to last element of a2
a2[-2]

We can update `ndarray` in place by index


In [ ]:
a2[-1] = 0
a2[-1]

Slicing returns subarrays of the original `ndarray`. Crucially it does this inexpensively by returning a "view" on the data rather than copying it. This is much more efficient and relies on numpy is using it's knowledge of the memory layout to return only the things you ask for. This is a general tactic used by numpy, if it _can_ return a view rather than a copy it _will_! If you really need a copy, try the `.copy()` method on `ndarrays`.

Slicing uses the same notation as core python: `[start:stop:step]` where `start` is inclusive and `stop` is exclusive.

In [ ]:
# Values between 3 and 19 of a2
a2[3:19]

In [ ]:
# Every third value between 3 and 19
a2[3:19:3]

Using negative is allowed for all three parts of the slice, but for the step you have to think a bit

In [ ]:
# Values between -10 and -2
a2[-10:-2]

Notice that the first argument of the slice is still inclusive and the second is not. If we omit a value when specifying the slice `start` defaults to 0, `end` defaults to the last element and `step` defaults to 1.

In [ ]:
a2[-2:-10:-1]

In [ ]:
a2[::3]

For multi-dimensional arrays the indexing notation is similar

In [ ]:
b = np.arange(100)
b.shape = (10, 10)
b

How about row 0, column 3 (remember python is 0 indexed)

In [ ]:
b[0,3]

In [ ]:
b[:, -1]

Or the fifth column of the first two rows

In [ ]:
b[0:2, 4]

In [ ]:
b[0:2, 4:6]

In [ ]:
b[::2, 9]

## Reshaping Arrays

Sometimes it is convenient to reshape arrays. I did this above by setting the `.shape` attribute but numpy arrays also have a reshape method.

In [ ]:
c = np.arange(27)
c

`reshape` expects a sequence as the first argument (e.g. a tuple) so

In [ ]:
d = c.reshape((3, 9))
d

Reshaping isn't enough to provoke numpy to copy the data, all it needs to do is make a new view on the same data

In [ ]:
d.base is c

Another common reshaping task is to add dimension(s) to an existing array. numpy has a special `newaxis` object for this task. This is a powerful idea when combined with numpy's broadcasting rules (see below).

In [ ]:
e = np.arange(10)
# 10 x 1
e[:, np.newaxis]

In [ ]:
# 1 x 10
e[np.newaxis,:]

In [ ]:
# 1 x 1 x 10
e[np.newaxis, np.newaxis, :]

## Stacking & Splitting ndarrays

You can combine general ndarrays with the `np.concatenate` and split them with `np.split`. There are also a number of convenience methods for commonly used shapes.

 * `hstack`
 * `vstack`
 * `hsplit`
 * `vsplit`

`hstack` is short for horizontal stack

In [ ]:
a = np.array([1, 2, 3, 4])
b = np.array([4, 3, 2, 1])

np.hstack((a, b))

For `2D` arrays this means we are joining columns

In [ ]:
# 4 x 1 & 4 x 1
np.hstack((a[np.newaxis, :].T, b[np.newaxis, :].T))

In [ ]:
# 1 x 4 & 
# 1 x 4
np.vstack((a, b))

In [ ]:
# 4 x 1 & 
# 4 x 1
np.vstack((a[:, np.newaxis], b[:, np.newaxis]))

In higher dimensions `concatenate` or `stack` should do what you need, but you need to manually tell it which axis to use for the stacking.

`np.split` goes in the opposite direction. It will try to produce sub-arrays of equal size. Again there are `hsplit` and `vsplit` variants for common use cases.

In [ ]:
np.split(np.arange(64).reshape((8, 8)), 2, axis=0)

In [ ]:
np.split(np.arange(64).reshape((8, 8)), 2, axis=1)

## Universal Functions (ufuncs)

The real reason for using numpy is so you can do numerical operations, _quickly_. Python uses a concept called `ufuncs` or universal function. A ufunc is a function which operates on `ndarrays` element-by-element. More formally, a `ufunc` is a vectorized wrapper around a function which can do a transformation on an `ndarray` and produces another `ndarray`. This element by element behaviour is fundamentally different from the usual python behaviour.

The key to writing fast numeric python code is: **Avoid for & while loops as far as you can, use numpy ufuncs as far as possible**


Lets start with basic arithmetic operations. Numpy can use it's internal broadcasting to do these quickly and efficiently

The usual operations are available

  * +: addition
  * -: subtraction
  * *: multiplication
  * /: division
  * //: integer division
  * **: power operator
  * %: modulo

Remember operations are element by element, and you can build up more complicated expressions as you go

In [ ]:
la = np.linspace(0, 1, 100)

(la ** 2 + la) / (la + 1)

**Example**: Try to calculate the terms of this sum as an `ndarray`
$$
\sqrt{12}\sum_{k=0}^{10}\frac{(-3)^{-k}}{2k+1}
$$

_Hints_: Start with `np.arange(11)` and think term by term, summing at the end

The operators we were using `+,-,/,...` actually correspond to functions (`ufuncs`)

|operator|function|description|
|--------|--------|-----------|
| + | np.add | Addition |
| - | np.subtract | Subtraction |
| - | np.negative | Unary negation |
| * | np.multiply | Multiplication |
| / | np.division | Ordinary floating point division |
| // | np.floor_divide | floor (integer) division |
| % | np.mod | Modulo/Remainder division |

You can use either syntax, but in the function notation there are lots more functions to play with e.g.

| function | description |
|----------|-------------|
| np.sin   | sin function |
| np.cos   | cos function |
| np.tan   | tan function |
| np.abs   | absolute value |
| np.exp   | exponential |
| np.log   | natural log |
| np.log2  | log base 2 |
| np.log10 | log base 10 |
|  ...     |    ...      |


In [ ]:
p1 = np.linspace(0, 2*np.pi, 25)
p2 = np.sin(p1)
p2

# p2 sin of p1

In [ ]:
q1 = np.linspace(0, 1, 10) + np.linspace(1, 2, 10)*1j
q1

In [ ]:
np.abs(q1)

## Aggregate Functions

Aggregate functions take an `ndarray` and reduce it along one (or more) axes. An example would be taking an array of numbers and calculating the mean value...

In [ ]:
r1 = np.linspace(0, 10, 100)
r1.mean()

But there are lots of aggregate functions

  * `min`: Minumum value
  * `max`: Maximum value
  * `sum`: Sum values
  * `prod`: Product of values
  * `mean`: Arthmetic mean
  * `std`: Standard deviation
  * `var`: Variance
  * `argmin`: indices of the minimum value
  * `argmax`: indices of the maximum value
  * `all`: is a condition true in all elements
  * `any`: is a condition true in any elements
  * `allclose`: All the values are within a small tolerance **really useful!**
  
  The default is to reduce along all axes, if you want to reduce along a specific axis you can pass that as an argument (the axes you specify are the ones which get squashed) 

In [ ]:
s1 = np.arange(50)
s2 = s1.reshape(5,10)
s2

In [ ]:
s2.mean(axis=1)

In [ ]:
s2.argmax(axis=0)

In [ ]:
np.argmax(s2, axis=0)

In [ ]:
np.unravel_index(s2.argmax(), s2.shape)

For binary operations (e.g. addition) you can also do reduction, so starting from

[1, 3, 5, 7, 9]

`np.add.reduce` will add 1 to 3, add that to 5 and so on, ...

In [ ]:
t1 = np.arange(1, 10, 2)
np.add.reduce(t1)

## Fancy Indexing

Fancy is the idea of using another array of indices to access your data, it is useful when the combinations you want to select become a bit more complicated than you can handle with slicing. A typical case would be where you are evaulating some condition for all elements (e.g. t1 < 5). It allows a lot of flexibility, but the downside of that flexibility is it will usually return a copy of the array rather than a view of the original data.

In [ ]:
v1=np.arange(27)[::-1]
v1

In [ ]:
v1[[1, 4, 6]]

In [ ]:
v2 = v1.reshape(3,9)
v2

In higher dimensions think of zipping together the arguements, e.g. 0-th row, 4th column

In [ ]:
v2[[0, 1, 1], [3, 7, 8]]

In [ ]:
v2 > 10

In [ ]:
v2[v2 > 10]

*N.B. Fancy indexing usually creates copies of the `ndarray` because you usually can't reconstruct the selection with simple algebra*

## Broadcasting

`numpy` is at it's most efficient when it is operating element by element, but not all arrays are the same size. To work around this, `numpy` implements a set of rules under the name of `broadcasting` to make `ndarray`s conform whenever possible. This is great news; it means you don't have to worry about doing that yourself, but it is important to understand the rules so that you know how `numpy` will behave when combining differnt shaped `nbdarrays`. To get the idea, think of
```python
np.arange(10) * 5
```
`numpy` wants to operate element-by-element, but `5` isn't an `ndarray`, it's just a number. If we could prompte 5 to be a `1d` narray and put 5's in in every place `numpy` would be happy. This is the basic idea of broadcasting, in summary

1. Given two arrays of different dimensions, prepend 1, to the shape of the smaller array
1. Dimensions of size 1 are repeated (without copying)


In [ ]:
a = np.arange(15)
a = a.reshape(3, 5)
a

In [ ]:
b = np.arange(5)
b

If I want to multiply these two `ndarray`s, `b` has the smaller dimensions (1 vs. 2) so a dimension of length one will be prepended to `b`. `b` will then be repeated 3 times to conform with the shape of a.

In [ ]:
a * b

Just to be explicit, that operation does something lie

In [ ]:
btmp = np.repeat(b[np.newaxis, :], 3, axis=0)
a *  btmp

## Random

Numpy has a few important submodules but `np.random` is probably the most important. As you might expect, it lets you work with random numbers but in addition to simple random numer generators, it can sample from different distributions (30+ available), handle permutations and do lots of other handy things.

`numpy.random` uses the concept of a `Generators` to implement sampling. The idea is you create a generator object then call methods on that generator to sample from the various distributions. The original `Generator` will normally get it's entropy from a (hopefully reliable) source then you can keep asking it for the `__next__` random elements distributed however you need.

(The generator interface to `numpy.random` is relatively recent, occasionally you will still see me explicity call functions of the submodule because I'm a slow learner).

In [ ]:
rng = np.random.default_rng(47)

Here we've seeded the `Generator` with a specific value so the results are reporoducible but normally you would just call `np.random.default_rng()` to get a random value from the OS. Now we can sample from various distributions

In [ ]:
rng.normal(5.0, 1.0, (64, 64))

The binomial distribution

In [ ]:
rng.binomial(10, .5, size=20)

5 random integers between 10 and 20

In [ ]:
rng.integers(10, 20, 5)

`np.random` also has functions for shuffling arrays in-place (`np.shuffle`) and selecting elements at random from `ndarrays` (`np.choice`)

In [ ]:
a = np.arange(10)
np.random.shuffle(a)
a

In [ ]:
a = np.arange(10)
np.random.choice(a, 3, replace=False)

You can also weight the selections in `np.choice` with probabilities. e.g. Here are the letter frequencies of ordinary english text.

In [ ]:
letter_freq = {' ': 0.19266420666588144,
 'e': 0.09680968984305797,
 't': 0.07140241019840815,
 'a': 0.06361581392196947,
 'o': 0.06183938572048604,
 'i': 0.05349452953695084,
 'n': 0.0521037201730283,
 'h': 0.051232447485652234,
 's': 0.049280151278754014,
 'r': 0.04524648142979075,
 'd': 0.03375374929612461,
 'l': 0.03124157971419029,
 'u': 0.02392934301198968,
 'm': 0.021518821910249234,
 'w': 0.020208685943305965,
 'c': 0.02004895261728702,
 'f': 0.016262143363080305,
 'y': 0.016250849087503207,
 'g': 0.013559584564274916,
 'p': 0.012933559003715817,
 ',': 0.012259129404969158,
 '.': 0.01200420147051468,
 'b': 0.011160357738111564,
 'v': 0.0076220225466009876,
 'k': 0.006392559976636984,
 'x': 0.001353699601312072,
 'j': 0.0008260955850676769,
 'q': 0.0006663622590487316,
 'z': 0.00031946665203789066
}

We can use those relative frequency values as probabilities (they sum to ~1) and generate a random sample of letters

In [ ]:
letters = np.array(list(letter_freq.keys()))
probabilities = np.array(list(letter_freq.values()))
chosen = np.random.choice(letters, 1000, p=probabilities)
''.join(chosen)

### Footnotes
[1]<span id="fn1"> If you dig deep enough in some of the numpy/scipy code you will find that the actual instructions you are executing were compiled from fortran and C. In general you can pass things quite easily between existing libraries and python, but fortran and C use different storage orders for multi-dimensional arrays so you have to be a little bit careful_.</span>